# 第一题

In [2]:
import numpy as np
import colour
from scipy.optimize import minimize, LinearConstraint, Bounds, NonlinearConstraint

# 获取 BT.2020 色彩空间对象
bt2020 = colour.RGB_COLOURSPACES['ITU-R BT.2020']
srgb = colour.RGB_COLOURSPACES['sRGB']

# 随机采样 RGB（属于 BT.2020 空间的 RGB 向量）
rgb_bt2020 = np.random.rand(1000, 3)
print(rgb_bt2020.shape)

# 将 RGB（BT.2020）转换为 XYZ
xyz_bt2020 = colour.RGB_to_XYZ(
    rgb_bt2020,
    bt2020.whitepoint,
    bt2020.whitepoint,
    bt2020.matrix_RGB_to_XYZ
)

# 将 XYZ 转换为 sRGB 空间的 RGB
rgb_srgb = colour.XYZ_to_RGB(
    xyz_bt2020,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)

# 可选：裁剪到 sRGB 显示范围 [0, 1]
rgb_srgb_clipped = np.clip(rgb_srgb, 0, 1)

# 检查有多少点本来就在 sRGB 色域内
in_gamut = np.all((rgb_srgb >= 0) & (rgb_srgb <= 1), axis=1)
print(f"{np.sum(in_gamut)} / {len(rgb_srgb)} colors are in sRGB gamut.")

def objective(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T  # Apply M
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    lab_target = colour.XYZ_to_Lab(xyz_bt2020)
    lab_result = colour.XYZ_to_Lab(xyz_srgb)
    delta_E = colour.delta_E(lab_target, lab_result, method='CIE 2000')
    return np.mean(delta_E)

def gamut_constraint(M_flat):
    M = M_flat.reshape(3, 3)
    xyz_srgb = (M @ xyz_bt2020.T).T
    rgb_srgb = colour.XYZ_to_RGB(
        xyz_srgb,
        srgb.whitepoint,
        srgb.whitepoint,
        srgb.matrix_XYZ_to_RGB
    )
    # 返回距离越界的最小 margin（负数代表不合法）
    return np.min(np.minimum(rgb_srgb, 1 - rgb_srgb))

# 初始猜测为单位矩阵
M0 = np.eye(3).flatten()

# 非线性约束：所有 RGB 分量必须 ∈ [0, 1]
nonlinear_constraint = NonlinearConstraint(gamut_constraint, -0.01, np.inf)
# 原本的约束条件过于严苛了，这里放宽了标准
print(f"deltaE: {objective(M0)}")

result = minimize(objective, M0, constraints=[nonlinear_constraint])
print(f"Optimization success: {result.success}")
print(f"Optimized matrix M:\n{result.x.reshape(3, 3)}")
print(f"deltaE: {result.fun}")
print(f"Optimization message: {result.message}")

# 获取色彩空间
bt2020 = colour.RGB_COLOURSPACES['ITU-R BT.2020']
srgb = colour.RGB_COLOURSPACES['sRGB']

# 使用已经优化好的矩阵M
M = result.x.reshape(3, 3)

# 在BT.2020 RGB空间中均匀采样
samples = 100000
np.random.seed(42)  # 设置随机种子以确保结果可复现
rgb_bt2020 = np.random.rand(samples, 3)

# 将RGB(BT.2020)转换为XYZ
xyz_bt2020 = colour.RGB_to_XYZ(
    rgb_bt2020,
    bt2020.whitepoint,
    bt2020.whitepoint,
    bt2020.matrix_RGB_to_XYZ
)

# 优化前：直接将BT.2020 XYZ转换为sRGB
rgb_srgb_direct = colour.XYZ_to_RGB(
    xyz_bt2020,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)
in_gamut_before = np.all((rgb_srgb_direct >= 0) & (rgb_srgb_direct <= 1), axis=1)
coverage_before = np.sum(in_gamut_before) / samples * 100

# 优化后：应用优化矩阵M，然后转换为sRGB
xyz_transformed = (M @ xyz_bt2020.T).T
rgb_srgb_optimized = colour.XYZ_to_RGB(
    xyz_transformed,
    srgb.whitepoint,
    srgb.whitepoint,
    srgb.matrix_XYZ_to_RGB
)
in_gamut_after = np.all((rgb_srgb_optimized >= 0) & (rgb_srgb_optimized <= 1), axis=1)
coverage_after = np.sum(in_gamut_after) / samples * 100

# 输出结果
print(f"采样点数: {samples}")
print(f"优化前色域覆盖率: {coverage_before:.2f}%")
print(f"优化后色域覆盖率: {coverage_after:.2f}%")
print(f"覆盖率变化: {coverage_after - coverage_before:.2f}%")

(1000, 3)
470 / 1000 colors are in sRGB gamut.
deltaE: 0.0
Optimization success: True
Optimized matrix M:
[[ 0.46399751  0.42276109  0.07146956]
 [-0.1500842   1.0950407   0.03757578]
 [-0.03122394  0.13222089  0.88200622]]
deltaE: 6.360714136254783
Optimization message: Optimization terminated successfully
采样点数: 100000
优化前色域覆盖率: 49.64%
优化后色域覆盖率: 96.20%
覆盖率变化: 46.56%
